## MazeRunner DataAnalyse


In [4]:
import os
import pyarrow.feather as feather

In [2]:
os.getcwd()

os.chdir('../experiments')

# os.chdir("../nodes")

os.getcwd()

'D:\\Git\\MazeRunner\\experiments'

#### avr_speed_different_policies

Bij deze analyse laten wij de verschilden snelheden zien van de policies om de uitgang te vinden

In [5]:
df = feather.read_feather('avr_speed_different_policies/avr_speed_different_policies.feather')

In [11]:
df

,time,found_exit,n_alive,explored
0,138,1,4,"[[[False, False, False, False, False, False, F..."
1,138,0,4,"[[[True, True, True, True, True, True, True, T..."
2,138,0,4,"[[[True, True, True, True, True, True, True, T..."
3,150,3,4,"[[[False, False, False, False, False, False, F..."
4,150,3,4,"[[[False, False, False, False, False, False, F..."
5,150,3,4,"[[[False, False, False, False, False, False, F..."
6,150,3,4,"[[[False, False, False, False, False, False, F..."
7,150,3,4,"[[[False, False, False, False, False, False, F..."
8,150,3,4,"[[[False, False, False, False, False, False, F..."
9,150,3,4,"[[[False, False, False, False, False, False, F..."
